In [1]:
import os
import random
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam

from kerashypetune import KerasGridSearch

In [2]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

np.random.seed(33)
subset_train = np.random.uniform(0,1, len(y_train))
subset_test = np.random.uniform(0,1, len(y_test))

x_train = x_train[subset_train < 0.1] / 255
y_train = y_train[subset_train < 0.1]

x_test = x_test[subset_test < 0.3] / 255
y_test = y_test[subset_test < 0.3]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((6036, 28, 28), (6036,), (2963, 28, 28), (2963,))

# Multi Output

In [4]:
def get_model(param):
    
    set_seed(33)
    
    inp = Input((28,28))
    x = Flatten()(inp)
    
    enc = Dense(param['unit_1'], activation='relu')(x)
    enc = Dense(param['unit_2'], activation='relu')(enc)
    
    hidden = Dense(param['unit_hid'], activation='relu')(enc)
    
    dec = Dense(param['unit_2'], activation='relu')(hidden)
    dec = Dense(param['unit_1'], activation='relu')(dec)
    
    ae_out = Dense(28*28, activation='relu')(dec)
    ae_out = Reshape((28,28))(ae_out)
    
    class_out = Dense(10, activation='softmax')(hidden)
    
    model = Model(inp, [ae_out,class_out])
    model.compile(optimizer=Adam(learning_rate=param['lr']), 
                  loss=['mse','sparse_categorical_crossentropy'])
    
    return model

In [5]:
param_grid = {
    'unit_1': [128,64], 
    'unit_2': [64,32],
    'unit_hid' : [32,16],
    'lr': [1e-2,1e-3], 
    'epochs': 100, 
    'batch_size': 512
}

In [6]:
es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_loss', mode='auto', restore_best_weights=True)

hypermodel = get_model

kgs = KerasGridSearch(hypermodel, param_grid, monitor='val_loss', greater_is_better=False, tuner_verbose=1)
kgs.search(x_train, [x_train,y_train], validation_data=(x_test, [x_test,y_test]), callbacks=[es])


16 trials detected for ('unit_1', 'unit_2', 'unit_hid', 'lr', 'epochs', 'batch_size')

***** (1/16) *****
Search({'unit_1': 128, 'unit_2': 64, 'unit_hid': 32, 'lr': 0.01, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
SCORE: 0.22427 at epoch 7

***** (2/16) *****
Search({'unit_1': 128, 'unit_2': 64, 'unit_hid': 32, 'lr': 0.001, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
SCORE: 0.2332 at epoch 24

***** (3/16) *****
Search({'unit_1': 128, 'unit_2': 64, 'unit_hid': 16, 'lr': 0.01, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
SCORE: 0.26374 at epoch 7

***** (4/16) *****
Search({'unit_1': 128, 'unit_2': 64, 'unit_hid': 16, 'lr': 0.001, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00029: early stopping
SCORE: 0.23569 at

<kerashypetune.KerasGridSearch>

In [7]:
print(kgs.scores)

[0.22427, 0.2332, 0.26374, 0.23569, 0.25271, 0.23084, 0.29183, 0.2333, 0.24856, 0.23422, 0.26177, 0.25652, 0.24808, 0.24891, 0.2828, 0.26808]


In [8]:
kgs.best_score

0.22427

In [9]:
kgs.best_params

{'unit_1': 128,
 'unit_2': 64,
 'unit_hid': 32,
 'lr': 0.01,
 'epochs': 7,
 'batch_size': 512,
 'steps_per_epoch': 12}

In [10]:
kgs.best_model

# Multi Input

In [11]:
def create_pairs(X,y, n_pairs=1):

    pairs, targets = [], []

    for i in range(len(y)):
        for _ in range(n_pairs):
            compare_to = i
            while compare_to == i:
                compare_to = random.randint(0,len(y)-1)
            pairs.append([X[i],X[compare_to]])
            if y[i] == y[compare_to]: # They are the same
                targets.append(1.)
            else:# Not the same
                targets.append(0.)
                
    pairs = np.asarray(pairs).transpose(1,0,2,3)
    targets = np.asarray(targets)

    return pairs, targets

In [12]:
np.random.seed(33)
random.seed(33)

train_pairs, train_y = create_pairs(x_train, y_train)
print(train_pairs.shape, train_y.shape)

test_pairs, test_y = create_pairs(x_test, y_test)
print(test_pairs.shape, test_y.shape)

(2, 6036, 28, 28) (6036,)
(2, 2963, 28, 28) (2963,)


In [13]:
def get_model(param):
    
    set_seed(33)
    
    left_input = Input((28,28))
    right_input = Input((28,28))

    shared = Sequential([
        Flatten(),
        Dense(param['unit_1'], activation=param['activ']),
        Dense(param['unit_2'], activation=param['activ']),
    ])

    encoded_l = shared(left_input)
    encoded_r = shared(right_input)

    L1_layer = Lambda(lambda tensor: tf.abs(tensor[0] - tensor[1]))

    L1_distance = L1_layer([encoded_l, encoded_r])
    out = Dense(1, activation='sigmoid')(L1_distance)
    
    model = Model([left_input,right_input], out)
    model.compile(optimizer=Adam(lr=param['lr']), 
                  loss="binary_crossentropy",
                  metrics=['accuracy'])
    
    return model

In [14]:
param_grid = {
    'unit_1': [128,64], 
    'unit_2': [64,32],
    'activ' : ['relu','elu'],
    'lr': [1e-2,1e-3], 
    'epochs': 100, 
    'batch_size': [256,512]
}

In [15]:
es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_loss', mode='auto', restore_best_weights=True)

hypermodel = get_model

kgs = KerasGridSearch(hypermodel, param_grid, monitor='val_loss', greater_is_better=False, tuner_verbose=1)
kgs.search([train_pairs[0],train_pairs[1]], train_y, validation_data=([test_pairs[0],test_pairs[1]], test_y), callbacks=[es])


32 trials detected for ('unit_1', 'unit_2', 'activ', 'lr', 'epochs', 'batch_size')

***** (1/32) *****
Search({'unit_1': 128, 'unit_2': 64, 'activ': 'relu', 'lr': 0.01, 'epochs': 100, 'batch_size': 256})
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
SCORE: 0.18378 at epoch 3

***** (2/32) *****
Search({'unit_1': 128, 'unit_2': 64, 'activ': 'relu', 'lr': 0.01, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
SCORE: 0.18168 at epoch 6

***** (3/32) *****
Search({'unit_1': 128, 'unit_2': 64, 'activ': 'relu', 'lr': 0.001, 'epochs': 100, 'batch_size': 256})
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
SCORE: 0.16768 at epoch 12

***** (4/32) *****
Search({'unit_1': 128, 'unit_2': 64, 'activ': 'relu', 'lr': 0.001, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
SCORE: 0.17568 

<kerashypetune.KerasGridSearch>

In [16]:
print(kgs.scores)

[0.18378, 0.18168, 0.16768, 0.17568, 0.1787, 0.17906, 0.18686, 0.19279, 0.18653, 0.18344, 0.18323, 0.17574, 0.17126, 0.17232, 0.19072, 0.20002, 0.18135, 0.17467, 0.18343, 0.1851, 0.1742, 0.1817, 0.19459, 0.19545, 0.1916, 0.19075, 0.19666, 0.19022, 0.17806, 0.1804, 0.20165, 0.20478]


In [17]:
kgs.best_score

0.16768

In [18]:
kgs.best_params

{'unit_1': 128,
 'unit_2': 64,
 'activ': 'relu',
 'lr': 0.001,
 'epochs': 12,
 'batch_size': 256,
 'steps_per_epoch': 24}

In [19]:
kgs.best_model